In [1]:
import pandas as pd
import numpy as np
import shutil
import copy
import os

In [2]:
imgFreq = {}
#,'183_1H','183_3H','183_7H'
#,'CPAC','EPAC','IO','SHEM','WPAC'
freqList = ['19H','19V','19','22V','37V','37H','37','91H','91V','91','150H','183_1H','183_3H','183_7H']
regList = ['ATL','CPAC','EPAC','IO','SHEM','WPAC']
funList = ['agglo']

for reg in regList:
    imgFreq[reg] = {}
    for freq in freqList:
        imgFreq[reg][freq] = {}
        for fun in funList:
            path = "..//..//AllFrequencies//"+reg+"//"+freq+"//"+fun+"//"
            imgFreq[reg][freq][fun] = []
            ls = os.listdir(path)
            for test in ls:
                if test == "Test_14":
                    imgFreq[reg][freq][fun].append( pd.read_csv(path+test+"//testInfo.csv") )

In [58]:
def PasteImages( df,path ):
    df = copy.deepcopy(df[df.SilhouetteVal > 0])
    
    for i,r in df.iterrows():
        cL_path = path+str(r.ClusterLabel)+"//"
        '''
        +str(r.T_No)
        if os.path.isdir(tNo_path) == False:
            os.mkdir(tNo_path)
        tNo_path+="//"
        '''
        imgName = r.FileName.split("//")[9]
        
        actualFile = cL_path+imgName
        #shutil.copy( r.FileName,cL_path )
        t_no= str(r.T_No).replace(".","-")
        
        renameFile = cL_path+t_no+"_"+imgName[:len(imgName)-4]+".png"
        #print(actualFile)
        #print(renameFile)
        
        os.rename(actualFile,renameFile )
        #r.FileName = r.FileName[ :len(r.FileName)-4 ]+".png"
        

In [59]:
#for reg in regList:
    #for freq in freqList:

reg = 'ATL'
freq = '19'
path = "..//..//AllFrequencies//"+reg+"//"+freq+"//"+"agglo//Test_14//"
PasteImages( imgFreq[reg][freq]["agglo"][0], path )


Creating a Zip file of all the specific folders

In [48]:
from zipfile import ZipFile
reg = 'ATL'
freq = '91'
zipObj = ZipFile(reg+"_"+freq+"_agglo_test_14.zip", 'w')
path = "..//..//AllFrequencies//"+reg+"//"+freq+"//"+"agglo//Test_14//"
for folderName, subfolders, filenames in os.walk(path):
    for filename in filenames:
        #create complete filepath of file in directory
        filePath = os.path.join(folderName, filename)
        print(filePath)
        zipObj.write(filePath)
zipObj.close()

..//..//AllFrequencies//ATL//91//agglo//Test_14//dendogram_lastp_18.json
..//..//AllFrequencies//ATL//91//agglo//Test_14//dendrogram_lastp_18.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//testInfo.csv
..//..//AllFrequencies//ATL//91//agglo//Test_14//0\3-5_20080718T092300_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//0\5-5_20080905T114800_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//0\2-5_20100831T103700_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//0\2-5_20100831T092900_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//0\4-0_20100917T102100_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//0\2-5_20100923T090700_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//0\1-0_20110812T095900_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//0\3-0_20110830T204600_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//0\2-0_20110907T081100_SSMIS_F17.png
..//..//AllFrequencies//ATL

..//..//AllFrequencies//ATL//91//agglo//Test_14//1\4-0_20170917T220300_SSMIS_F18.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//1\3-0_20180903T062600_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//1\3-5_20180903T092800_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//1\2-5_20180816T100100_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//1\4-0_20180909T210200_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//1\3-5_20180909T201400_SSMIS_F18.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//1\4-0_20181012T091000_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//1\3-0_20190826T080000_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//1\3-0_20190825T222100_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//1\6-0_20190830T225800_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//1\3-0_20190910T082000_SSMIS_F18.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//2\3-0_20091004T0

..//..//AllFrequencies//ATL//91//agglo//Test_14//5\4-0_20101030T115900_SSMIS_F18.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//5\1-0_20110811T214900_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//5\8-5_20110811T071000_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//5\1-0_20110812T103000_SSMIS_F18.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//5\4-0_20110903T213900_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//5\4-0_20110904T000300_SSMIS_F18.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//5\3-5_20120621T090000_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//5\4-0_20120621T111100_SSMIS_F18.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//5\3-0_20120805T075600_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//5\3-0_20120805T075300_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//5\1-0_20120808T201100_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//5\1-0_20120808T2

..//..//AllFrequencies//ATL//91//agglo//Test_14//7\3-5_20181009T095000_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//7\8-5_20181023T183400_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//7\4-0_20181028T215900_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//7\4-0_20181028T220100_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//7\3-0_20190825T192300_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//7\6-0_20190831T112700_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//7\6-0_20190831T103200_SSMIS_F18.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//7\8-5_20190901T065800_SSMIS_F18.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//7\4-0_20190925T063100_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//7\4-0_20190925T091700_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//8\4-5_20080710T120100_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//8\4-0_20080710T2

..//..//AllFrequencies//ATL//91//agglo//Test_14//9\4-0_20120915T091000_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//9\3-5_20121001T221000_SSMIS_F18.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//9\3-0_20130910T094600_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//9\4-0_20140918T082800_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//9\4-0_20140918T111800_SSMIS_F18.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//9\3-5_20141013T103000_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//9\5-0_20151004T120500_SSMIS_F18.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//9\5-0_20151004T112300_SSMIS_F19.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//9\4-0_20151111T112000_SSMIS_F19.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//9\2-0_20160823T105200_SSMIS_F18.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//9\1-0_20160908T102300_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//9\5-0_20161012T2

..//..//AllFrequencies//ATL//91//agglo//Test_14//11\1-0_20110809T203300_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//11\2-5_20110802T123100_SSMIS_F18.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//11\2-0_20110814T095000_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//11\1-0_20110812T232700_SSMIS_F18.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//11\3-0_20110914T112700_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//11\1-0_20110926T103700_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//11\3-0_20111002T093300_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//11\3-0_20111002T205100_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//11\3-0_20111002T120100_SSMIS_F18.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//11\3-0_20111002T231900_SSMIS_F18.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//11\3-0_20111111T105500_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//11\1-

..//..//AllFrequencies//ATL//91//agglo//Test_14//12\2-5_20120628T221600_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//12\2-5_20120629T003300_SSMIS_F18.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//12\2-0_20120806T203700_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//12\2-0_20120806T203400_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//12\2-0_20120809T224100_SSMIS_F18.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//12\1-0_20120813T205100_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//12\1-0_20120813T204700_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//12\1-0_20120820T105800_SSMIS_F18.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//12\2-0_20120826T194400_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//12\3-5_20120901T201500_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//12\3-5_20120901T200900_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//12\2-

..//..//AllFrequencies//ATL//91//agglo//Test_14//13\2-0_20120828T110000_SSMIS_F18.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//13\3-5_20120904T095700_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//13\3-0_20120910T101800_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//13\3-5_20120904T095200_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//13\3-0_20120910T101300_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//13\4-0_20120906T235000_SSMIS_F18.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//13\4-0_20120916T201600_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//13\2-5_20121003T214500_SSMIS_F18.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//13\2-5_20121004T083700_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//13\2-0_20121003T084400_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//13\2-5_20121004T083000_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//13\2-

..//..//AllFrequencies//ATL//91//agglo//Test_14//15\4-0_20100919T225600_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//15\2-0_20100912T080500_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//15\1-0_20110819T085000_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//15\3-5_20111110T110900_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//15\2-0_20120623T195900_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//15\3-0_20120805T103700_SSMIS_F18.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//15\1-0_20120811T193400_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//15\3-0_20120919T194600_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//15\3-0_20120919T081600_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//15\3-0_20120919T193900_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//15\3-0_20131120T192100_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//15\2-

..//..//AllFrequencies//ATL//91//agglo//Test_14//17\1-0_20120824T071600_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//17\1-0_20120824T071200_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//17\1-0_20120829T075400_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//17\1-0_20120829T074900_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//17\1-0_20121001T073700_SSMIS_F16.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//17\1-0_20121008T203100_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//17\1-0_20130723T095500_SSMIS_F18.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//17\3-0_20131118T093100_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//17\8-5_20140728T101600_SSMIS_F18.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//17\3-5_20141011T105400_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//17\3-5_20141011T221200_SSMIS_F17.png
..//..//AllFrequencies//ATL//91//agglo//Test_14//17\8-

In [14]:
from zipfile import ZipFile

for reg in regList:
    zipObj = ZipFile(reg+'agglo_test_14.zip', 'w')
    for freq in freqList:
        path = "..//..//AllFrequencies//"+reg+"//"+freq+"//"+"agglo//Test_14//"
        for folderName, subfolders, filenames in os.walk(path):
            for filename in filenames:
                #create complete filepath of file in directory
                filePath = os.path.join(folderName, filename)
                zipObj.write(filePath)
    zipObj.close()

Code Below aims at finding common images among multiple frequencies

In [ ]:
def Test( df ):
    df = copy.deepcopy(df[df.SilhouetteVal > 0])
    imgCol = []
    for i,r in df.iterrows():
        imgCol.append( r.FileName.split("//")[9]  )
    df['ImgName'] = imgCol 
    return df

def CompareTest( df1, df2 ):
    df = df1.merge(df2, left_on='ImgName', right_on='ImgName')
    
    return pd.DataFrame({"ImgName":df.ImgName})

In [ ]:
finalDf = {}
for reg in regList:
    finalDf[reg] = Test( imgFreq[reg][freqList[0]]['agglo'][0] )
    for i in range(1,len(freqList)):
        finalDf[reg] = CompareTest( finalDf[reg], Test(imgFreq[reg][freqList[i]]['agglo'][0]) )

In [ ]:
finalDf['WPAC']

In [ ]:
df1 = Test( imgFreq['ATL']['19H']['kmeans'][9] )
df2 = Test( imgFreq['ATL']['91H']['kmeans'][9] )
df3 = Test( imgFreq['ATL']['150H']['kmeans'][9] )
df4 = Test( imgFreq['ATL']['183_1H']['kmeans'][9] )

df_imgName = CompareTest(df1,df2)
df_imgName = CompareTest(df_imgName, df3)
df_imgName = CompareTest(df_imgName, df4)

df1 = df1.merge( df_imgName, left_on='ImgName', right_on='ImgName' )
df2 = df2.merge( df_imgName, left_on='ImgName', right_on='ImgName' )
df3 = df3.merge( df_imgName, left_on='ImgName', right_on='ImgName' )
df4 = df4.merge( df_imgName, left_on='ImgName', right_on='ImgName' )

In [ ]:
def LabelAndSill(df,silVal, maxLabel):
    dr= df[ df.T_No == silVal ]
    print("-------------------------")
    for i in range(maxLabel):
        print(len( dr[ dr.ClusterLabel == i] ) )
    ''' 
    j=0
    while j <9.5:
        dr=  df[ df.T_No == j ]
        maxLab = -1
        maxVal = -1
        for i in range(maxLabel):
            if len(dr[dr.ClusterLabel == i]) > maxVal:
                maxVal = len(dr[dr.ClusterLabel == i])
                maxLab = i

        print(j,maxVal,maxLab )
        #dr[dr.ClusterLabel == maxLab]
        j+=0.5
    '''

In [ ]:
reg = 'ATL'
freq = '183_1H'
fun = 'kmeans'
j = 9
path = "..//..//AllFrequencies//"+reg+"//"+freq+"//"+fun+"//Test_"+str(j)+"//"
for i,r in df4.iterrows():
    tNo_path = path+str(r.ClusterLabel)+"//"+str(r.T_No)
    if os.path.isdir(tNo_path) == False:
        os.mkdir(tNo_path)
    shutil.copy( r.FileName,tNo_path )
    


In [ ]:
from matplotlib import pyplot as plt
import matplotlib.image as mpimage

w=10
h=10
fig=plt.figure(figsize=(8, 8))
columns = 3
rows = 7
for i in range(1, columns*rows +1):
    img = np.random.randint(10, size=(h,w))
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
plt.show()

In [ ]:
reg = 'ATL'
freq = '19H'
fun = 'kmeans'
test = 9
df = imgFreq[reg][freq][fun][test]
df = df[ df.SilhouetteVal>0.0 ]


'''
imgCol = []
for i,r in df.iterrows():
    imgCol.append( r.FileName.split("//")[9]  )
df['ImgName'] = imgCol
'''



In [ ]:
dr = df[ df.T_No == 1.0 ]
dr[ dr.ClusterLabel == 0 ].FileName.iloc[3]

In [ ]:
'''
def Ans(df):
    
    uniqueId = sorted(list(df.ClusterLabel_x.unique()))
    for label in uniqueId:
        tno[label] = {}
        df1 = df.T_No[df.ClusterLabel == label]
        t1 = list(df1)
        i=0
        while i <9.5:
            tno[label][i] = t1.count(i)
            i+=0.5
    
    newDf = pd.DataFrame(tno)
    print(newDf)


df = df[ df.Sil]

'''


    
    '''imgCol = []
    for i,r in df3.iterrows():
        imgCol.append( r.FileName.split("//")[9]  )
    df3['ImgName'] = imgCol   
    
    if (df.T_No_x == df.T_No_y).all():
        if (df.T_No_x == df.T_No).all():
            print("aplee")
    
            
    
    
    
    return df
    '''    
    
    

    '''
    label1 = {}
    for i,r in t1.iterrows():
        if label1.get(r.ClusterLabel) == None:
            label1[r.ClusterLabel] = []
        label1[r.ClusterLabel].append([r.FileName, r.SilhouetteVal, r.T_No])
    
    label2 = {}
    for i,r in t2.iterrows():
        if label2.get(r.ClusterLabel) == None:
            label2[r.ClusterLabel] = []
        label2[r.ClusterLabel].append([r.FileName, r.SilhouetteVal, r.T_No])
    
    for k1,v1 in label1.items():
        for k2,v2 in label2.items():
            
            #v1 and v2 are the lists of rows for a particular label
            count=0
            for l1 in v1:
                for l2 in v2:
                    if l1[0] == l2[0]:
                        count+=1
            print(k1,k2,count)
    '''

In [ ]:
df1 = imgFreq['ATL']['150H']['agglo'][6]
df1 = df1[df1.SilhouetteVal > 0]

In [ ]:
newdf1 = df1.T_No[ df1.ClusterLabel == 4]
t1 = list(newdf1)
i=0
while i <9.5:
    print(i, t1.count(i))
    i+=0.5
print(t1)   

In [ ]:
newdf1 = df2.T_No[ df2.ClusterLabel == 6 ]
t1 = list(newdf1)
i=0
while i <9.0:
    print(i, t1.count(i))
    i+=0.5

In [ ]:
for i in range(11):
    print( len( df1[ (df1['ClusterLabel']==i) ] ) )

for i in range(7):
    print( len( df[ (df['ClusterLabel_x']==i) ] ) )
    
for i in range(7):
    print( len( imgFreq['ATL']['91H']['fuzzykmeans'][4][ (imgFreq['ATL']['91H']['fuzzykmeans'][9]['ClusterLabel']==i) ] ) )